In [1]:
import gmplot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import peakutils
import scipy
import os
import cv2
from PIL import Image

In [2]:
import pandas as pd
data = pd.read_csv('18-Aug-2021.csv')
data.shape

(104, 84)

In [3]:
R = pd.DataFrame()
R['Accx'] = data['Acceleration Sensor(X axis)(g)']
R['Accy'] = data['Acceleration Sensor(Y axis)(g)']
R['Accz'] = data['Acceleration Sensor(Z axis)(g)']
R['Lat'] = data[' Latitude']
R['Longi'] = data[' Longitude']
R['Speed'] = data['Speed (OBD)(km/h)']
R['Time'] = data['Trip Time(Since journey start)(s)']

R = R[R['Time']>0]
print(R.head())

        Accx      Accy      Accz        Lat      Longi  Speed  Time
19  0.999096  0.031812 -0.295734  18.630658  73.789709      4     1
20  1.024235  0.049383 -0.206897  18.630658  73.789709      7     2
21  0.857542 -0.035548 -0.002865  18.630658  73.789709      7     3
22  0.807756 -0.059222 -0.198110  18.630658  73.789709      7     4
23  1.035705  0.065491 -0.097071  18.630658  73.789709      7     5


### Using the filter on acceleration values

In [4]:
def average(list):
    # list1 = [w.replace('Acceleration Sensor(Z axis)(g)', '0') for w in list]
    list2 = [float(i) for i in list]
    res = 0
    for ele in list2:
        res += ele
        avg = res / 3
    res = [prod(list2[x:x + 3]) for x in range(0, len(list2), 3)]
    return res
def highpassfilter(Accelerometer_values):
    fc = 0.2
    b = 0.16
    N = int(np.ceil((4 / b)))
    if not N % 2: N += 1
    # Make sure that N is odd.
    n = np.arange(N)

    # Compute a low-pass filter.
    h = np.sinc(2 * fc * (n - (N - 1) / 2))
    w = np.blackman(N)
    h = h * w
    h = h / np.sum(h)

    # Create a high-pass filter from the low-pass filter through spectral inversion
    h = -h
    h[(N - 1) // 2] += 1
    s = np.convolve(Accelerometer_values, h)
    np.set_printoptions(suppress=False)
    s1 = s.astype(
        float)  # amplifying the output by 10 times for further computation
    return s1

### Creating the separate dataframe for pothole and speed breaker

In [5]:
pothole = R[R['Time']>0]
pothole = R[R['Accx']<0.7]
print(pothole.head())

        Accx      Accy      Accz        Lat      Longi  Speed  Time
39  0.487796  0.105760 -0.325021  18.630707  73.790384     24    21
45  0.688655  0.300519 -0.317698  18.630682  73.790826     25    27
72  0.684995 -0.114134 -0.361873  18.630295  73.792155     14    54


In [6]:
Speed_breaker = R[R['Time']>0]
Speed_breaker = R[R['Accx']>1.3]
print(Speed_breaker.head())

        Accx      Accy      Accz        Lat      Longi  Speed  Time
29  1.496242 -0.236895 -0.233500  18.630673  73.789886     13    11
48  1.356640 -0.006993 -0.176635  18.630653  73.791026     26    30


### Getting time instances for the road anomalies

In [7]:
def get_index(df):
    time_index = []
    for i in range(df.shape[0]):
        v = df['Speed'].iloc[i] * (5/18)
        x = df['Time'].iloc[i] - (14.34 / v)
        x = round(x)*2
        time_index.append(x)
    return time_index

In [8]:
time_pothole = get_index(pothole)
print(time_pothole)
time_speed_breaker = get_index(Speed_breaker)
print(time_speed_breaker)

[38, 50, 100]
[14, 56]


### Geoplotting the anomaly instances

In [10]:
# for pothole 
P = []
for i in time_pothole:
    x = int(i/2)
    P.append(x)
print(P)

# for speed breaker
S = []
for i in time_speed_breaker:
    y = int(i/2)
    S.append(y)
print(S)

[19, 25, 50]
[7, 28]


In [11]:
def get_locations(df, X):
    Lat = []
    Long = []
    for i in range(len(X)):
        x = df['Lat'].iloc[X[i]]
        y = df['Longi'].iloc[X[i]]
        Lat.append(x)
        Long.append(y)
    return Lat, Long

In [13]:
potholes_lat, potholes_long = get_locations(R, P)
speed_breaker_lat, speed_breaker_long = get_locations(R, S)

In [15]:
def PlotMap(latitude_potholes, longitude_potholes, latitude_speed_breaker, longitude_speed_breaker):
    gmap = gmplot.GoogleMapPlotter(R['Lat'].iloc[1], R['Longi'].iloc[1], 17)
    gmap.scatter(latitude_potholes, longitude_potholes, 'red', size=6, marker=False)
    gmap.scatter(latitude_speed_breaker, longitude_speed_breaker, 'yellow', size=6, marker=False)
    gmap.draw("road_anomalies_1.html")

In [16]:
PlotMap(potholes_lat, potholes_long, speed_breaker_lat, speed_breaker_long)

### Converting video into frames

In [18]:
vidcap = cv2.VideoCapture('Pothole_trial_18_6_0.mp4')

In [19]:
def set_video(video):
    def getFrame(x):
        video.set(cv2.CAP_PROP_POS_MSEC, x * 1000)
        hasFrames, image = video.read()
        if hasFrames:
            #cv2.imwrite("image" + str(count) + ".jpg", image)  # save frame as JPG file
            cv2.imwrite(str(count) + ".jpg", image)
        return hasFrames

    sec = 0
    frameRate = 0.5  # //it will capture image in each 0.5 second
    count = 1
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

In [20]:
set_video(vidcap)

In [24]:
path = os.getcwd()
ImageDetected = [f for f in os.listdir(path) if f.endswith('.jpg')]

In [25]:
images = ImageDetected

lst = []
for k in images:
    temp = k.split('.')
    first = temp[0]
    lst.append(int(first))

lst.sort()

imagetemp = []
for k in lst:
    imagetemp.append(str(k) + ".jpg")
    
#print(imagetemp)
images = imagetemp

# print(images)

### Getting probable anomalies images from list of images 

In [26]:
def get_images(time):
    image = []
    for i in time:
        image.append(images[i-4:i])
    #rint(image_pothole)
    x = len(image)
    y = len(image[0])
    all_images = []
    for i in range(x):
        for j in range(y):
            all_images.append(image[i][j])

    return all_images

In [39]:
pothole_images = get_images(time_pothole)

In [30]:
speed_breaker_images = get_images(time_speed_breaker)

### Saving iamges to their locations

In [32]:
path_pothole = "D://G Drive backup//LTTS Internship//Reimplementing_codes//Pothole detection//Pothole_images//"
path_speed_breaker = "D://G Drive backup//LTTS Internship//Reimplementing_codes//Pothole detection//Speed_breaker_images//"

In [33]:
def save(images, path):
    for i in range(len(images)):
        mac = Image.open(images[i])
        mac.save(path+'Image'+str(i)+'.jpg')

In [34]:
save(pothole_images, path_pothole)

In [35]:
save(speed_breaker_images, path_speed_breaker)